In [42]:
# import libraries
import pandas as pd
import pinecone
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os
import concurrent.futures

In [43]:
# Load environment variables from a .env file
load_dotenv()

# Retrieve the Pinecone API key from environment variables
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("PINECONE_API_KEY not found in environment variables. Please set it in your .env file.")


In [44]:
# Load the CSV files into dataframes (as provided in your query)
inventory_df = pd.read_csv("datasets/inventory_data.csv", encoding="ISO-8859-1")
transport_df = pd.read_csv("datasets/transport_history.csv", encoding="ISO-8859-1")

In [45]:
inventory_df.head()

,ItemID,BatchNumber,GenericName,MaxInventory,CurrentStock,ReorderPoint,Unit,StorageCondition,SpecialHandling,UnitCost,SellingPrice,ManufacturingDate,ExpiryDate,LeadTimeDays,Status,LastUpdated
0,ITEM-CCEE944C,P2410-2692,Paracetamol,1000000,951453,93150.684932,g,Room Temperature,NaN,63.53,78.69,2024-10-18,2026-12-14,27,Active,2025-02-11 22:24:02
1,ITEM-32C437D9,I2410-7657,Ibuprofen,800000,770922,61369.863014,g,Room Temperature,NaN,46.62,82.27,2024-10-23,2027-10-14,21,Active,2025-02-11 22:24:02
2,ITEM-00DBC34C,A2408-6011,Aspirin,750000,194347,98630.136986,g,Room Temperature,NaN,87.73,137.51,2024-08-21,2026-02-02,41,Active,2025-02-11 22:24:02
3,ITEM-CC350352,A2409-4578,Atorvastatin,250000,16052,15753.424658,g,Room Temperature,NaN,64.53,148.80,2024-09-10,2027-02-23,16,Active,2025-02-11 22:24:02
4,ITEM-2AB2ABEF,L2412-6204,Lisinopril,300000,133725,28767.123288,g,Room Temperature,NaN,95.87,53.80,2024-12-28,2026-02-10,28,Active,2025-02-11 22:24:02


In [46]:
transport_df.head()

,ShipmentID,GenericName,CargoUnit,OriginLocationName,OriginCountry,OriginPort,OriginPortCode,DestinationLocationName,DestinationCountry,DestinationPort,...,MonitoringLevel,InsuranceType,InsuranceCoverage,InsurancePremium,HandlingEquipmentCost,HandlingLaborCost,TotalHandlingCost,CarbonFootprint,CarbonFootprintUnit,RequiredDocuments
0,SHIP-68722A1D,Growth Hormone,IU,Shanghai Facility,China,Shanghai Port,CNSHA,PharmaCo Manufacturing Plant,USA,Port of New York and New Jersey,...,Enhanced,Premium,0.144,0.01,0.00,0.00,0.00,0.00,kg CO2e,"['Commercial Invoice', 'Packing List', 'Certif..."
1,SHIP-CDA1A448,Lisinopril,g,Mexico City Facility,Mexico,Veracruz Port,MXVER,PharmaCo Manufacturing Plant,USA,Port of New York and New Jersey,...,Enhanced,Premium,7590.864,303.63,35.21,16.93,52.15,6.39,kg CO2e,"['Commercial Invoice', 'Packing List', 'Certif..."
2,SHIP-F41C9477,Lisinopril,g,Ho Chi Minh City Facility,Vietnam,Saigon Port,VNSGN,PharmaCo Manufacturing Plant,USA,Port of New York and New Jersey,...,Continuous,Standard,6297.290,283.38,25.65,9.01,34.66,183.46,kg CO2e,"['Commercial Invoice', 'Packing List', 'Certif..."
3,SHIP-24570857,Sildenafil,g,Montreal Facility,Canada,NaN,CAMTR,PharmaCo Manufacturing Plant,USA,Port of New York and New Jersey,...,Standard,Premium,4537.716,151.26,10.21,5.58,15.79,1.60,kg CO2e,"['Commercial Invoice', 'Packing List', 'Certif..."
4,SHIP-8F55442C,Beta Interferon,mcg,Shanghai Facility,China,Shanghai Port,CNSHA,PharmaCo Manufacturing Plant,USA,Port of New York and New Jersey,...,Continuous,Standard,0.000,0.00,0.00,0.00,0.00,0.00,kg CO2e,"['Commercial Invoice', 'Packing List', 'Certif..."


In [47]:
# concat both Combine the datasets into a single knowledge base
data = pd.concat([inventory_df, transport_df], ignore_index=True)

In [48]:
# Reset index to avoid potential KeyErrors
data = data.reset_index(drop=True)

# Clean TemperatureRange column to remove encoding issues
if 'TemperatureRange' in data.columns:
    data['TemperatureRange'] = data['TemperatureRange'].astype(str).apply(lambda x: x.encode('latin1').decode('utf-8', 'ignore'))


In [49]:
# Convert each row to a string representation
text_data = data.astype(str).apply(lambda x: ' | '.join(x), axis=1)

In [50]:
# Load Hugging Face Sentence Transformer Model
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [51]:
# Initialize Pinecone
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = "medical-supply-chain"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # 384 for MiniLM
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index(index_name)

In [52]:
# Function to upsert data in parallel with NaN handling
def upsert_batch(start_idx, batch_size):
    batch = text_data.iloc[start_idx: start_idx + batch_size]
    metadata_batch = data.iloc[start_idx: start_idx + batch_size].fillna("").to_dict(orient='records')  # Replace NaN
    embeddings = embed_model.encode(batch.tolist()).tolist()
    vectors = [
        {"id": str(start_idx + j), "values": embeddings[j], "metadata": metadata_batch[j]}
        for j in range(len(batch))
    ]
    index.upsert(vectors)

# Embed and Store Data in Pinecone using Parallel Processing
batch_size = 100  # Adjust batch size as needed
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(upsert_batch, i, batch_size) for i in range(0, len(text_data), batch_size)]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        future.result()

print("Data successfully stored in Pinecone!")

100%|██████████| 10/10 [01:20<00:00,  8.09s/it]

Data successfully stored in Pinecone!
